In [4]:
# libraries
import sys 
from copy import deepcopy

In [5]:
# parameters
depth = 3 

In [3]:
def extract_string(board_string):
    """ Extract the board state and the last move from the input string
    Args:
        board_string: the input string as specified in the guidelines
    
    Returns:
        current board state (python list), last_move (tuple(int))
    """
    
    i = 0 
    # extracting board
    board = []
    while board_string[i] != "L":
        if board_string[i] == "[":
            temp = []
        elif board_string[i] == "]":
            board.append(temp)
        else:
            temp.append(int(board_string[i]))
        i += 1

    # extracting last move 
    if board_string[i+9] == "n":
        last_move = None 
    else:
        temp_m = [int(s) for s in (board_string[i+12:i+17]).split(",")]
        last_move = tuple(temp_m)
    
    return board, last_move

In [ ]:
# class board for applying moves, determining available moves and evaluating the board.

class Board:
    
    def __init__(self, size):
        self.size = size
    
    def _find_zeros(self, board_state):
        pass 
    
    def _get_surround(self, board_state, move):
        pass 
    
    def available_moves(self, board_state, last_move):
        """ Computes all the available moves given the board_state and the previous move
        Returns:
            List of tuple(int): all the valid locations of moves.
        """
        if last_move !=  None:
            
        
    def evaluate(self, board_state, move):
        num_choices = len(self.available_moves(board_state, move))
        return num_choices if (num_choices < 6) else 6
        
    def has_lost(self, board_state, move):
        return False
    
    def apply_move(self, board_state, move):
        

In [ ]:
def min_max_alpha_beta(board, board_state, last_move, side, max_depth, 
                       alpha=-sys.float_info.max, beta=sys.float_info.max):
    """ Runs the minimax with alpha beta pruning 
    Inputs:
        board: Board()
        board_state: list of states
        last_move: (c, a, b)
        side (int): +1 or -1
        max_depth (int)
        alpha, beta: ignored
    Returns: 
        score, best_move
    """
    best_score_move = None
    
    moves = board.available_moves(board_state, last_move)
    
    # if no available moves, return tied with no moves 
    if not moves:
        return 0, None
    
    for move in moves:
        new_board_state = board.apply_move(board_state, move, side)
        
        # if the game is lost in this move 
        if board.has_lost(new_board_state, move):
            score = -10000
        else:
            # stopping search if depth has reached
            if max_depth <= 1:
                score = board.evaluate(new_board_state, move)
            else:
                score, _ = min_max_alpha_beta(board, new_board_state, move, -side, 
                                              max_depth - 1, evaluation_func, alpha,
                                              beta)
        if side > 0:
            if score > alpha:
                alpha = score
                best_score_move = move
        else:
            if score < beta:
                beta = score
                best_score_move = move
        if alpha >= beta:
            break

    return alpha if side > 0 else beta, best_score_move


In [ ]:
if __name__ == "__main__":
    
    cur_board_state, last_move = extract_string(sys.argv[1])
    
    # New Board 
    board = Board(len(cur_board_state)-2) 
    
    
    
    
    
    
